In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pytorch_lightning as pl
from src import Model, ViT, DataModule, parse_config_file

In [10]:
config_file = 'vit.yml'
config = parse_config_file(config_file)
config

{'lr': 0.001,
 'optimizer': 'Adam',
 'batch_size': 256,
 'scheduler': {'LambdaLR': {'lr_lambda': 'lambda epoch: 1.0 / (1.0 + epoch)'}},
 'data': 'data_extra',
 'subset': 0.1,
 'num_workers': 0,
 'pin_memory': 1,
 'backbone': 'vit_base_patch16_384',
 'pretrained': 1,
 'unfreeze': 0,
 'size': 384,
 'train_trans': {'RandomResizedCrop': {'height': 384, 'width': 384},
  'Transpose': {},
  'HorizontalFlip': {},
  'VerticalFlip': {},
  'ShiftScaleRotate': {},
  'Normalize': {}},
 'val_trans': {'CenterCrop': {'height': 384, 'width': 384}, 'Normalize': {}},
 'gpus': 1,
 'precision': 16,
 'max_epochs': 15,
 'val_batches': 5,
 'es_start_from': 5,
 'patience': 3}

In [11]:
dm = DataModule(file = config['data'], **config)

model = ViT(config)

In [ ]:
trainer = pl.Trainer(
    gpus=config['gpus'],
    precision=config['precision'],
    limit_val_batches=config['val_batches'],
    auto_scale_batch_size='binsearch'
)

trainer.tune(model, dm)

In [12]:
model.hparams.batch_size = 32
model.hparams

"backbone":      vit_base_patch16_384
"batch_size":    32
"data":          data_extra
"es_start_from": 5
"gpus":          1
"lr":            0.001
"max_epochs":    15
"num_workers":   0
"optimizer":     Adam
"patience":      3
"pin_memory":    1
"precision":     16
"pretrained":    1
"scheduler":     {'LambdaLR': {'lr_lambda': 'lambda epoch: 1.0 / (1.0 + epoch)'}}
"size":          384
"subset":        0.1
"train_trans":   {'RandomResizedCrop': {'height': 384, 'width': 384}, 'Transpose': {}, 'HorizontalFlip': {}, 'VerticalFlip': {}, 'ShiftScaleRotate': {}, 'Normalize': {}}
"unfreeze":      0
"val_batches":   5
"val_trans":     {'CenterCrop': {'height': 384, 'width': 384}, 'Normalize': {}}

In [13]:
trainer = pl.Trainer(
    gpus=config['gpus'],
    precision=config['precision'],
    limit_val_batches=config['val_batches'],
    auto_lr_find=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [14]:
lr_finder = trainer.tuner.lr_find(model, dm)

Training samples:  21642
Validation samples:  5411
Training only on 2165 samples



  | Name  | Type              | Params
--------------------------------------------
0 | model | VisionTransformer | 86.1 M
--------------------------------------------
86.1 M    Trainable params
0         Non-trainable params
86.1 M    Total params


RuntimeError: CUDA out of memory. Tried to allocate 1.91 GiB (GPU 0; 24.00 GiB total capacity; 21.08 GiB already allocated; 92.47 MiB free; 21.55 GiB reserved in total by PyTorch)

In [ ]:
fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
new_lr = lr_finder.suggestion()
model.hparams.lr = new_lr
model.hparams.lr

In [ ]:
#trainer.fit(model, dm)